# 🎬 Automated Biceps Curl False Form Dataset Pipeline

Bu notebook otomatik olarak:
1. **YouTube'dan** "biceps curl mistakes" videolarını indirir
2. **Mediapipe Pose** ile videoları analiz eder
3. `biceps_curl_form_analysis.ipynb` ile aynı feature'ları kullanarak **yanlış form segmentlerini** tespit eder
4. **FFmpeg** ile bu segmentleri kesip `false/` klasörüne kaydeder

---

## 1. Kütüphanelerin Kurulumu

In [ ]:
# Gerekli kütüphaneleri kur
!pip install mediapipe opencv-python numpy pandas tqdm -q

# yt-dlp'yi en son versiyona güncelle (ÖNEMLİ!)
!pip install --upgrade yt-dlp -q

# FFmpeg kurulumu (Colab için)
!apt-get update -qq && apt-get install -y ffmpeg -qq

print("✅ Kurulum tamamlandı!")

In [ ]:
# yt-dlp versiyonunu kontrol et
!yt-dlp --version

In [ ]:
# Import'lar
import os
import subprocess
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
from typing import List, Tuple, Dict, Optional
from dataclasses import dataclass
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

# Mediapipe Pose başlat
mp_pose = mp.solutions.pose

print("✅ Import'lar tamamlandı!")
print(f"OpenCV: {cv2.__version__}")
print(f"Mediapipe: {mp.__version__}")

## 2. Konfigürasyon

In [ ]:
# =====================================================
# KONFIGÜRASYON
# =====================================================

# YouTube indirme ayarları
YOUTUBE_QUERIES = [
    "biceps curl mistakes",
    "biceps curl wrong form",
    "biceps curl common errors",
]
MAX_VIDEOS_PER_QUERY = 3

# Klasör yapısı
RAW_VIDEO_DIR = "/content/raw_videos"
FALSE_OUTPUT_DIR = "/content/dataset/train/false"

# Video analiz parametreleri
FRAME_SKIP = 2
MIN_SEGMENT_SECONDS = 2.0
MAX_SEGMENT_SECONDS = 6.0

# =====================================================
# YANLIŞ FORM TESPİT EŞİKLERİ
# =====================================================

@dataclass
class FormThresholds:
    """Yanlış form tespiti için eşik değerleri"""
    torso_angle_max: float = 15.0
    torso_angle_std_max: float = 5.0
    elbow_rom_min: float = 80.0
    elbow_min_angle_max: float = 60.0
    angular_velocity_percentile: float = 85.0
    shoulder_y_std_max: float = 0.03
    shoulder_x_std_max: float = 0.02

THRESHOLDS = FormThresholds()

print("✅ Konfigürasyon ayarlandı!")

## 3. Yardımcı Fonksiyonlar

In [ ]:
def calculate_angle(point_a, point_b, point_c) -> float:
    """
    Üç nokta arasındaki açıyı hesaplar. (biceps_curl_form_analysis.ipynb ile aynı)
    """
    a = np.array(point_a[:2])
    b = np.array(point_b[:2])
    c = np.array(point_c[:2])
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    angle = np.degrees(np.arccos(cosine_angle))
    
    return float(angle)


def calculate_vertical_angle(shoulder, hip) -> float:
    """Torso açısı hesaplama"""
    vertical_point = (shoulder[0], shoulder[1] - 1.0)
    return calculate_angle(vertical_point, shoulder, hip)


print("✅ Açı hesaplama fonksiyonları hazır!")

## 4. YouTube Video İndirme (Düzeltilmiş Versiyon)

In [ ]:
def download_youtube_videos_v2(queries: List[str], max_per_query: int, output_dir: str) -> List[str]:
    """
    yt-dlp ile YouTube'dan video indirir.
    Düzeltilmiş versiyon: daha basit format seçimi, daha iyi hata yönetimi.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for query in queries:
        print(f"\n🔍 Aranıyor: '{query}' (max {max_per_query} video)")
        
        # Basit ve güvenilir komut
        cmd = [
            "yt-dlp",
            f"ytsearch{max_per_query}:{query}",
            "-f", "best[ext=mp4]/best",  # En iyi mp4 veya herhangi bir format
            "-o", os.path.join(output_dir, "%(id)s.%(ext)s"),
            "--no-playlist",
            "--restrict-filenames",
            "--no-overwrites",
            "--socket-timeout", "30",
        ]
        
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=600)
            
            if result.returncode == 0:
                print(f"   ✅ '{query}' için indirme tamamlandı")
            else:
                # Hata mesajını göster
                error_msg = result.stderr[:300] if result.stderr else "Bilinmeyen hata"
                print(f"   ⚠️ '{query}' için uyarı:")
                print(f"      {error_msg}")
                
        except subprocess.TimeoutExpired:
            print(f"   ⏱️ '{query}' için zaman aşımı (10 dakika)")
        except Exception as e:
            print(f"   ❌ '{query}' için hata: {e}")
    
    # İndirilen dosyaları listele
    downloaded_files = []
    if os.path.exists(output_dir):
        for f in os.listdir(output_dir):
            if f.lower().endswith(('.mp4', '.mkv', '.avi', '.mov', '.webm')):
                downloaded_files.append(os.path.join(output_dir, f))
    
    print(f"\n✅ Toplam {len(downloaded_files)} video indirildi.")
    return downloaded_files


print("✅ YouTube indirme fonksiyonu hazır!")

In [ ]:
# Alternatif: Direkt URL ile indirme
def download_from_urls(video_urls: List[str], output_dir: str) -> List[str]:
    """
    Spesifik YouTube URL'lerinden video indirir.
    Arama çalışmazsa bu fonksiyonu kullanın.
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for url in video_urls:
        print(f"\n⬇️ İndiriliyor: {url}")
        
        cmd = [
            "yt-dlp",
            url,
            "-f", "best[ext=mp4]/best",
            "-o", os.path.join(output_dir, "%(id)s.%(ext)s"),
            "--restrict-filenames",
            "--no-overwrites",
        ]
        
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=300)
            if result.returncode == 0:
                print(f"   ✅ İndirme tamamlandı")
            else:
                print(f"   ⚠️ Hata: {result.stderr[:200]}")
        except Exception as e:
            print(f"   ❌ Hata: {e}")
    
    # İndirilen dosyaları listele
    downloaded_files = []
    if os.path.exists(output_dir):
        for f in os.listdir(output_dir):
            if f.lower().endswith(('.mp4', '.mkv', '.avi', '.mov', '.webm')):
                downloaded_files.append(os.path.join(output_dir, f))
    
    return downloaded_files


print("✅ URL tabanlı indirme fonksiyonu hazır!")

In [ ]:
# =====================================================
# YÖNTEM 1: Arama ile indirme (yeni versiyon)
# =====================================================

SKIP_DOWNLOAD = False  # True yaparak atlayabilirsiniz

if not SKIP_DOWNLOAD:
    downloaded_videos = download_youtube_videos_v2(
        queries=YOUTUBE_QUERIES,
        max_per_query=MAX_VIDEOS_PER_QUERY,
        output_dir=RAW_VIDEO_DIR
    )
else:
    downloaded_videos = [
        os.path.join(RAW_VIDEO_DIR, f) 
        for f in os.listdir(RAW_VIDEO_DIR) 
        if f.lower().endswith(('.mp4', '.mkv', '.avi', '.mov', '.webm'))
    ] if os.path.exists(RAW_VIDEO_DIR) else []
    print(f"⏭️ İndirme atlandı. Mevcut: {len(downloaded_videos)} video")

print(f"\n📹 İşlenecek video sayısı: {len(downloaded_videos)}")

In [ ]:
# =====================================================
# YÖNTEM 2: Direkt URL ile indirme (arama çalışmazsa)
# =====================================================

# Aşağıdaki URL'leri biceps curl hata videoları ile değiştirin
# https://www.youtube.com/watch?v=VIDEO_ID formatında

MANUAL_VIDEO_URLS = [
    # "https://www.youtube.com/watch?v=XXXX",
    # "https://www.youtube.com/watch?v=YYYY",
]

if MANUAL_VIDEO_URLS and len(downloaded_videos) == 0:
    print("📋 Manuel URL'lerden indirme yapılıyor...")
    downloaded_videos = download_from_urls(
        video_urls=MANUAL_VIDEO_URLS,
        output_dir=RAW_VIDEO_DIR
    )
    print(f"\n📹 İşlenecek video sayısı: {len(downloaded_videos)}")

## 5. Feature Extraction (biceps_curl_form_analysis.ipynb ile aynı)

In [ ]:
@dataclass
class FrameFeatures:
    """Tek frame için feature'lar"""
    frame_idx: int
    timestamp: float
    elbow_left_angle: float
    elbow_right_angle: float
    shoulder_left_x: float
    shoulder_left_y: float
    shoulder_right_x: float
    shoulder_right_y: float
    torso_angle: float


@dataclass
class SegmentFeatures:
    """Segment için özet feature'lar"""
    start_time: float
    end_time: float
    elbow_left_min: float
    elbow_left_max: float
    elbow_left_range: float
    elbow_left_mean: float
    elbow_left_std: float
    elbow_right_min: float
    elbow_right_max: float
    elbow_right_range: float
    elbow_right_mean: float
    elbow_right_std: float
    shoulder_left_x_std: float
    shoulder_left_y_std: float
    shoulder_right_x_std: float
    shoulder_right_y_std: float
    torso_angle_mean: float
    torso_angle_std: float
    torso_angle_max: float
    elbow_left_ang_vel_max: float
    elbow_right_ang_vel_max: float
    wrong_form_score: float = 0.0

print("✅ Dataclass'lar tanımlandı!")

In [ ]:
def extract_frame_features(video_path: str, pose_model, frame_skip: int = 2) -> List[FrameFeatures]:
    """Videodan frame bazlı feature'ları çıkarır."""
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"⚠️ Video açılamadı: {video_path}")
        return []
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        fps = 30.0
    
    # Landmark indeksleri
    LEFT_SHOULDER = mp_pose.PoseLandmark.LEFT_SHOULDER.value
    LEFT_ELBOW = mp_pose.PoseLandmark.LEFT_ELBOW.value
    LEFT_WRIST = mp_pose.PoseLandmark.LEFT_WRIST.value
    LEFT_HIP = mp_pose.PoseLandmark.LEFT_HIP.value
    RIGHT_SHOULDER = mp_pose.PoseLandmark.RIGHT_SHOULDER.value
    RIGHT_ELBOW = mp_pose.PoseLandmark.RIGHT_ELBOW.value
    RIGHT_WRIST = mp_pose.PoseLandmark.RIGHT_WRIST.value
    RIGHT_HIP = mp_pose.PoseLandmark.RIGHT_HIP.value
    
    frame_features = []
    frame_idx = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_idx % frame_skip != 0:
            frame_idx += 1
            continue
        
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose_model.process(image_rgb)
        
        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            
            try:
                l_shoulder = (lm[LEFT_SHOULDER].x, lm[LEFT_SHOULDER].y)
                l_elbow = (lm[LEFT_ELBOW].x, lm[LEFT_ELBOW].y)
                l_wrist = (lm[LEFT_WRIST].x, lm[LEFT_WRIST].y)
                l_hip = (lm[LEFT_HIP].x, lm[LEFT_HIP].y)
                
                r_shoulder = (lm[RIGHT_SHOULDER].x, lm[RIGHT_SHOULDER].y)
                r_elbow = (lm[RIGHT_ELBOW].x, lm[RIGHT_ELBOW].y)
                r_wrist = (lm[RIGHT_WRIST].x, lm[RIGHT_WRIST].y)
                r_hip = (lm[RIGHT_HIP].x, lm[RIGHT_HIP].y)
                
                elbow_left_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)
                elbow_right_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
                
                mid_shoulder = ((l_shoulder[0] + r_shoulder[0]) / 2, (l_shoulder[1] + r_shoulder[1]) / 2)
                mid_hip = ((l_hip[0] + r_hip[0]) / 2, (l_hip[1] + r_hip[1]) / 2)
                torso_angle = calculate_vertical_angle(mid_shoulder, mid_hip)
                
                ff = FrameFeatures(
                    frame_idx=frame_idx,
                    timestamp=frame_idx / fps,
                    elbow_left_angle=elbow_left_angle,
                    elbow_right_angle=elbow_right_angle,
                    shoulder_left_x=l_shoulder[0],
                    shoulder_left_y=l_shoulder[1],
                    shoulder_right_x=r_shoulder[0],
                    shoulder_right_y=r_shoulder[1],
                    torso_angle=torso_angle
                )
                frame_features.append(ff)
                
            except (IndexError, AttributeError):
                pass
        
        frame_idx += 1
    
    cap.release()
    return frame_features

print("✅ Frame extraction fonksiyonu hazır!")

In [ ]:
def calculate_segment_features(frames: List[FrameFeatures]) -> Optional[SegmentFeatures]:
    """Segment özet feature'larını hesaplar."""
    if len(frames) < 5:
        return None
    
    elbow_left = np.array([f.elbow_left_angle for f in frames])
    elbow_right = np.array([f.elbow_right_angle for f in frames])
    shoulder_left_x = np.array([f.shoulder_left_x for f in frames])
    shoulder_left_y = np.array([f.shoulder_left_y for f in frames])
    shoulder_right_x = np.array([f.shoulder_right_x for f in frames])
    shoulder_right_y = np.array([f.shoulder_right_y for f in frames])
    torso = np.array([f.torso_angle for f in frames])
    
    elbow_left_vel = np.abs(np.diff(elbow_left))
    elbow_right_vel = np.abs(np.diff(elbow_right))
    
    return SegmentFeatures(
        start_time=frames[0].timestamp,
        end_time=frames[-1].timestamp,
        elbow_left_min=np.min(elbow_left),
        elbow_left_max=np.max(elbow_left),
        elbow_left_range=np.max(elbow_left) - np.min(elbow_left),
        elbow_left_mean=np.mean(elbow_left),
        elbow_left_std=np.std(elbow_left),
        elbow_right_min=np.min(elbow_right),
        elbow_right_max=np.max(elbow_right),
        elbow_right_range=np.max(elbow_right) - np.min(elbow_right),
        elbow_right_mean=np.mean(elbow_right),
        elbow_right_std=np.std(elbow_right),
        shoulder_left_x_std=np.std(shoulder_left_x),
        shoulder_left_y_std=np.std(shoulder_left_y),
        shoulder_right_x_std=np.std(shoulder_right_x),
        shoulder_right_y_std=np.std(shoulder_right_y),
        torso_angle_mean=np.mean(torso),
        torso_angle_std=np.std(torso),
        torso_angle_max=np.max(torso),
        elbow_left_ang_vel_max=np.max(elbow_left_vel) if len(elbow_left_vel) > 0 else 0,
        elbow_right_ang_vel_max=np.max(elbow_right_vel) if len(elbow_right_vel) > 0 else 0,
    )

print("✅ Segment feature hesaplama hazır!")

In [ ]:
def calculate_wrong_form_score(segment: SegmentFeatures, thresholds: FormThresholds) -> float:
    """Yanlış form skorunu hesaplar (0-1 arası)."""
    score = 0.0
    max_score = 0.0
    
    # Torso açısı
    max_score += 0.25
    if segment.torso_angle_mean > thresholds.torso_angle_max:
        score += 0.25 * min(segment.torso_angle_mean / 30.0, 1.0)
    
    # Torso instabilitesi
    max_score += 0.15
    if segment.torso_angle_std > thresholds.torso_angle_std_max:
        score += 0.15 * min(segment.torso_angle_std / 10.0, 1.0)
    
    # Düşük ROM
    max_score += 0.20
    avg_rom = (segment.elbow_left_range + segment.elbow_right_range) / 2
    if avg_rom < thresholds.elbow_rom_min:
        score += 0.20 * (1 - avg_rom / thresholds.elbow_rom_min)
    
    # Min açı yüksek
    max_score += 0.15
    avg_min_angle = (segment.elbow_left_min + segment.elbow_right_min) / 2
    if avg_min_angle > thresholds.elbow_min_angle_max:
        score += 0.15 * min((avg_min_angle - thresholds.elbow_min_angle_max) / 40.0, 1.0)
    
    # Yüksek açısal hız
    max_score += 0.15
    max_vel = max(segment.elbow_left_ang_vel_max, segment.elbow_right_ang_vel_max)
    if max_vel > 20.0:
        score += 0.15 * min(max_vel / 50.0, 1.0)
    
    # Omuz instabilitesi
    max_score += 0.10
    avg_shoulder_y_std = (segment.shoulder_left_y_std + segment.shoulder_right_y_std) / 2
    if avg_shoulder_y_std > thresholds.shoulder_y_std_max:
        score += 0.10 * min(avg_shoulder_y_std / 0.1, 1.0)
    
    return min(score / max_score, 1.0) if max_score > 0 else 0.0

print("✅ Yanlış form skoru hesaplama hazır!")

In [ ]:
def find_wrong_form_segments(
    frame_features: List[FrameFeatures],
    thresholds: FormThresholds,
    min_segment_seconds: float = 2.0,
    max_segment_seconds: float = 6.0,
    score_threshold: float = 0.3,
    fps: float = 30.0
) -> List[Tuple[float, float, float]]:
    """Yanlış form segmentlerini bulur."""
    if len(frame_features) < 10:
        return []
    
    window_size = int(min_segment_seconds * fps / FRAME_SKIP)
    step_size = max(window_size // 2, 1)
    
    candidate_segments = []
    
    for start_idx in range(0, len(frame_features) - window_size, step_size):
        end_idx = min(start_idx + window_size * 2, len(frame_features))
        window_frames = frame_features[start_idx:end_idx]
        
        segment_feat = calculate_segment_features(window_frames)
        if segment_feat is None:
            continue
        
        score = calculate_wrong_form_score(segment_feat, thresholds)
        
        if score >= score_threshold:
            start_time = window_frames[0].timestamp
            end_time = min(window_frames[-1].timestamp, start_time + max_segment_seconds)
            candidate_segments.append((start_time, end_time, score))
    
    if not candidate_segments:
        return []
    
    # En yüksek skorlu, örtüşmeyen segmentleri seç
    candidate_segments.sort(key=lambda x: x[2], reverse=True)
    
    final_segments = []
    used_times = []
    
    for start, end, score in candidate_segments:
        overlaps = False
        for used_start, used_end in used_times:
            if not (end < used_start or start > used_end):
                overlaps = True
                break
        
        if not overlaps:
            final_segments.append((start, end, score))
            used_times.append((start, end))
    
    final_segments.sort(key=lambda x: x[0])
    return final_segments

print("✅ Segment bulma fonksiyonu hazır!")

## 6. FFmpeg ile Segment Kesme

In [ ]:
def cut_video_segment(video_path: str, start_time: float, end_time: float, output_path: str) -> bool:
    """FFmpeg ile video segmentini keser."""
    try:
        cmd = [
            "ffmpeg", "-y",
            "-i", video_path,
            "-ss", f"{start_time:.2f}",
            "-to", f"{end_time:.2f}",
            "-c:v", "libx264",
            "-c:a", "aac",
            "-loglevel", "error",
            output_path
        ]
        subprocess.run(cmd, check=True, capture_output=True)
        return True
    except subprocess.CalledProcessError as e:
        print(f"⚠️ FFmpeg hatası: {e}")
        return False

print("✅ FFmpeg kesme fonksiyonu hazır!")

## 7. Ana Pipeline

In [ ]:
def process_video(video_path: str, output_dir: str, pose_model, thresholds: FormThresholds, score_threshold: float = 0.3) -> int:
    """Tek videoyu işler."""
    os.makedirs(output_dir, exist_ok=True)
    
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0:
        fps = 30.0
    cap.release()
    
    frame_features = extract_frame_features(video_path, pose_model, FRAME_SKIP)
    
    if len(frame_features) < 10:
        print(f"   ⚠️ Yetersiz frame: {os.path.basename(video_path)}")
        return 0
    
    wrong_segments = find_wrong_form_segments(
        frame_features, thresholds,
        min_segment_seconds=MIN_SEGMENT_SECONDS,
        max_segment_seconds=MAX_SEGMENT_SECONDS,
        score_threshold=score_threshold,
        fps=fps
    )
    
    if not wrong_segments:
        print(f"   ℹ️ Segment bulunamadı: {os.path.basename(video_path)}")
        return 0
    
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    cut_count = 0
    
    for idx, (start, end, score) in enumerate(wrong_segments):
        output_name = f"{video_name}_false_{idx:02d}_score{score:.2f}.mp4"
        output_path = os.path.join(output_dir, output_name)
        
        if cut_video_segment(video_path, start, end, output_path):
            print(f"   ✅ {output_name} (score={score:.2f})")
            cut_count += 1
    
    return cut_count

print("✅ Video işleme fonksiyonu hazır!")

In [ ]:
def run_pipeline(video_paths: List[str], output_dir: str, thresholds: FormThresholds, score_threshold: float = 0.3) -> Dict:
    """Ana pipeline."""
    print("="*60)
    print("🎬 YANLIŞ FORM SEGMENT TESPİT PİPELINE'I")
    print("="*60)
    print(f"\n📊 Video sayısı: {len(video_paths)}")
    print(f"📁 Output: {output_dir}")
    print(f"🎯 Skor eşiği: {score_threshold}")
    
    stats = {'total': len(video_paths), 'processed': 0, 'segments': 0, 'failed': []}
    
    with mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        smooth_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as pose_model:
        
        for video_path in tqdm(video_paths, desc="İşleniyor"):
            print(f"\n📹 {os.path.basename(video_path)}")
            
            try:
                count = process_video(video_path, output_dir, pose_model, thresholds, score_threshold)
                stats['processed'] += 1
                stats['segments'] += count
            except Exception as e:
                print(f"   ❌ Hata: {e}")
                stats['failed'].append(video_path)
    
    print("\n" + "="*60)
    print("📊 SONUÇLAR")
    print("="*60)
    print(f"✅ İşlenen: {stats['processed']}/{stats['total']}")
    print(f"✅ Kesilen segment: {stats['segments']}")
    
    return stats

print("✅ Pipeline hazır!")

## 8. Pipeline'ı Çalıştır

In [ ]:
# Mevcut videoları listele
if os.path.exists(RAW_VIDEO_DIR):
    video_files = [
        os.path.join(RAW_VIDEO_DIR, f)
        for f in os.listdir(RAW_VIDEO_DIR)
        if f.lower().endswith(('.mp4', '.mkv', '.avi', '.mov', '.webm'))
    ]
    print(f"📹 Bulunan video: {len(video_files)}")
else:
    video_files = []
    print("⚠️ Raw video klasörü yok!")

In [ ]:
# Pipeline'ı çalıştır
if video_files:
    results = run_pipeline(
        video_paths=video_files,
        output_dir=FALSE_OUTPUT_DIR,
        thresholds=THRESHOLDS,
        score_threshold=0.25  # Daha düşük = daha fazla segment
    )
else:
    print("⚠️ İşlenecek video yok!")
    print("\n💡 İpuçları:")
    print("   1. yt-dlp'yi güncelleyin: !pip install --upgrade yt-dlp")
    print("   2. Manuel URL'ler kullanın (MANUAL_VIDEO_URLS listesi)")
    print("   3. Videoları manuel olarak raw_videos klasörüne yükleyin")

In [ ]:
# Çıktıları kontrol et
if os.path.exists(FALSE_OUTPUT_DIR):
    files = os.listdir(FALSE_OUTPUT_DIR)
    print(f"\n📁 False klasöründe {len(files)} dosya:")
    for f in sorted(files)[:15]:
        print(f"   - {f}")
else:
    print("⚠️ Henüz çıktı yok.")

---
## 📝 Özet

Bu notebook:
1. ✅ YouTube'dan biceps curl hata videoları indirir
2. ✅ `biceps_curl_form_analysis.ipynb` ile aynı feature'ları çıkarır
3. ✅ Yanlış form segmentlerini tespit eder
4. ✅ FFmpeg ile kesip `false/` klasörüne kaydeder